<a href="https://colab.research.google.com/github/MaInthiyaz/Codtech/blob/main/Data_analysis_Big_Data_Analysis_T1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Task1 Big data analysis(CT)



In [1]:
!pip install pyspark


Import PySpark and start Spark session

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigDataTask").getOrCreate()


large and suitable dataset for this task:

Dataset: New York City Taxi Trip Data
Size: Several GBs (monthly CSV files)
Why it fits: It's large-scale (millions of records), publicly available, and ideal for PySpark/Dask processing.
inferSchema=True: Automatically infers data types for each column.

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, desc

# Start a Spark session
spark = SparkSession.builder.appName("CodTech Big Data Analysis").getOrCreate()

# Load your dataset
df = spark.read.csv("data_reports_monthly.csv", header=True, inferSchema=True)

# Show schema to confirm column names
df.printSchema()


root
 |-- Month/Year: timestamp (nullable = true)
 |-- License Class: string (nullable = true)
 |-- Trips Per Day: string (nullable = true)
 |-- Farebox Per Day: string (nullable = true)
 |-- Unique Drivers: string (nullable = true)
 |-- Unique Vehicles: string (nullable = true)
 |-- Vehicles Per Day: string (nullable = true)
 |-- Avg Days Vehicles on Road: double (nullable = true)
 |-- Avg Hours Per Day Per Vehicle: double (nullable = true)
 |-- Avg Days Drivers on Road: double (nullable = true)
 |-- Avg Hours Per Day Per Driver: double (nullable = true)
 |-- Avg Minutes Per Trip: double (nullable = true)
 |-- Percent of Trips Paid with Credit Card: string (nullable = true)
 |-- Trips Per Day Shared: string (nullable = true)



Convert Numeric Columns from String to Double
Some columns like "Trips Per Day" and "Farebox Per Day" are read as strings. We need to cast them to numeric types for analysis.

In [11]:
# Cast key columns to double for math operations
df = df.withColumn("Trips Per Day", col("Trips Per Day").cast("double"))
df = df.withColumn("Farebox Per Day", col("Farebox Per Day").cast("double"))
df = df.withColumn("Percent of Trips Paid with Credit Card", col("Percent of Trips Paid with Credit Card").cast("double"))


Trip & Fare Summary

In [12]:
df.select("Trips Per Day", "Farebox Per Day").describe().show()


+-------+-----------------+---------------+
|summary|    Trips Per Day|Farebox Per Day|
+-------+-----------------+---------------+
|  count|               23|              0|
|   mean|611.4347826086956|           NULL|
| stddev|238.2891341678227|           NULL|
|    min|            148.0|           NULL|
|    max|            986.0|           NULL|
+-------+-----------------+---------------+



Trend Over Time (Month/Year)

In [13]:
df.groupBy("Month/Year").agg(
    avg("Trips Per Day").alias("Avg Trips"),
    avg("Farebox Per Day").alias("Avg Fare")
).orderBy("Month/Year").show()
# Shows how demand and revenue change monthly.

+-------------------+---------+--------+
|         Month/Year|Avg Trips|Avg Fare|
+-------------------+---------+--------+
|2010-01-01 00:00:00|     NULL|    NULL|
|2010-02-01 00:00:00|     NULL|    NULL|
|2010-03-01 00:00:00|     NULL|    NULL|
|2010-04-01 00:00:00|     NULL|    NULL|
|2010-05-01 00:00:00|     NULL|    NULL|
|2010-06-01 00:00:00|     NULL|    NULL|
|2010-07-01 00:00:00|     NULL|    NULL|
|2010-08-01 00:00:00|     NULL|    NULL|
|2010-09-01 00:00:00|     NULL|    NULL|
|2010-10-01 00:00:00|     NULL|    NULL|
|2010-11-01 00:00:00|     NULL|    NULL|
|2010-12-01 00:00:00|     NULL|    NULL|
|2011-01-01 00:00:00|     NULL|    NULL|
|2011-02-01 00:00:00|     NULL|    NULL|
|2011-03-01 00:00:00|     NULL|    NULL|
|2011-04-01 00:00:00|     NULL|    NULL|
|2011-05-01 00:00:00|     NULL|    NULL|
|2011-06-01 00:00:00|     NULL|    NULL|
|2011-07-01 00:00:00|     NULL|    NULL|
|2011-08-01 00:00:00|     NULL|    NULL|
+-------------------+---------+--------+
only showing top

Credit Card Payment Percentage

In [14]:
df.select("Month/Year", "Percent of Trips Paid with Credit Card") \
  .orderBy(desc("Percent of Trips Paid with Credit Card")) \
  .show(5)
#Displays the top 5 months with highest card payments.

+-------------------+--------------------------------------+
|         Month/Year|Percent of Trips Paid with Credit Card|
+-------------------+--------------------------------------+
|2025-03-01 00:00:00|                                  NULL|
|2025-03-01 00:00:00|                                  NULL|
|2025-03-01 00:00:00|                                  NULL|
|2025-02-01 00:00:00|                                  NULL|
|2025-02-01 00:00:00|                                  NULL|
+-------------------+--------------------------------------+
only showing top 5 rows



Driver Utilization

In [15]:
df.select("Avg Hours Per Day Per Driver", "Avg Days Drivers on Road").describe().show()
#Helps understand driver workload trends.

+-------+----------------------------+------------------------+
|summary|Avg Hours Per Day Per Driver|Avg Days Drivers on Road|
+-------+----------------------------+------------------------+
|  count|                         812|                     812|
|   mean|           5.204556650246314|      18.099137931034488|
| stddev|           1.924869089466435|      3.7050885455208946|
|    min|                         1.7|                     6.8|
|    max|                         8.5|                    23.8|
+-------+----------------------------+------------------------+



Vehicle Utilization

In [16]:
df.select("Avg Hours Per Day Per Vehicle", "Avg Days Vehicles on Road").describe().show()
#Similar stats for fleet management analysis.

+-------+-----------------------------+-------------------------+
|summary|Avg Hours Per Day Per Vehicle|Avg Days Vehicles on Road|
+-------+-----------------------------+-------------------------+
|  count|                          812|                      812|
|   mean|            6.174137931034492|       19.682881773399007|
| stddev|           3.5464778895615376|       5.4617007573035945|
|    min|                          1.8|                      6.7|
|    max|                         15.3|                     30.1|
+-------+-----------------------------+-------------------------+



Trip Duration Insight

In [17]:
df.select("Avg Minutes Per Trip").describe().show()
#Shows the efficiency of each ride.

+-------+--------------------+
|summary|Avg Minutes Per Trip|
+-------+--------------------+
|  count|                 812|
|   mean|  20.627832512315244|
| stddev|  18.024483656382174|
|    min|                 0.0|
|    max|                75.0|
+-------+--------------------+



| Deliverable Type | Example Result                             |
| ---------------- | ------------------------------------------ |
| Script           | PySpark script (as above)                  |
| Insights         | Monthly trend, usage patterns, utilization |
| Tools Used       | PySpark, DataFrame API                     |
| Dataset Used     | `data_reports_monthly.csv`                 |
